# active _learning.ipynb

A PASCAL VOC dataset structure is used. See https://docs.ultralytics.com/datasets/detect/voc/ for details.

In [1]:
from ultralytics import YOLO
import pandas as pd
import glob
import os
from icecream import ic
import sqlite3
import random
from datetime import datetime
import shutil
import subprocess

In [1]:
IMAGE_DIR_PATH = '/home/aubrey/Desktop/Guam07-training-set/rawdata'
DB_PATH = '/home/aubrey/Desktop/Guam07-training-set/code/active_learning2.sqlite3'
DATASET_PATH = '/home/aubrey/Desktop/Guam07-training-set/datasets/active_learning2'
LABELIMG_PATH = '/home/aubrey/labelImg/labelImg.py' 
MODEL_FILE_PATH = '/home/aubrey/Desktop/Guam07-training-set/code/runs/detect/imgsz9603/weights/best.pt'

In [3]:
def time_format(): 
    now = datetime.now() 
    return f'{now.strftime("%H:%M:%S")} --> '
  
ic.configureOutput(prefix=time_format) 

In [4]:
def create_db(image_dir_path: str, db_path: str):
    """
    Creates an SQLite database and adds an 'img' and an 'obj'.
    The default value for 'img.subset' is 'pool'.
    'img.randy' is populated with 'random.randint(0, 1000000)'. 
    Set a seed in the main code to enable reproducable random sequences (eg: 'random.seed(42)').
    """
    ic()
    conn = sqlite3.connect(db_path)
    # conn.row_factory = sqlite3.Row   # enables accessing values in results by field name
    conn.execute(
        '''CREATE TABLE img (
        filename TEXT PRIMARY KEY, 
        randy INTEGER, 
        subset TEXT DEFAULT "pool",
        edit_flag INTEGER DEFAULT 0
        );''')
    conn.commit()
    conn.execute(
        '''CREATE TABLE obj (
        filename TEXT,
        cls INTEGER,
        x REAL,
        y REAL,
        w REAL,
        h REAL,
        conf REAL        
        );''')
    conn.commit()
    for imagepath in glob.iglob(f'{image_dir_path}/*.jpg'):
        filename = os.path.basename(imagepath)
        randy = random.randint(0, 1000000)
        conn.execute(f'INSERT INTO img (filename, randy) VALUES("{filename}", {randy});')
    conn.commit()
    conn.close()
    
# random.seed(42)
# initialize_db(image_dir_path=IMAGE_DIR_PATH, db_path='temp.sqlite')   

In [5]:
def pool2subset(subset: str, n: int):
    '''
    Changes the images.subset DB field in "n" randomly selected records from "pool" to the value specified by subset.
    If "n" is larger than the number of records tagged as "pool", then all available "pool" images are converted to "subset" images.
    '''
    ic()
    conn = sqlite3.connect(DB_PATH)
    sql = f'''UPDATE img SET subset = "{subset}"
        WHERE subset = "pool"
        ORDER BY randy 
        LIMIT {n};'''
    conn.execute(sql)
    conn.commit()
    conn.close()

# # Adds 50 records to 'train000' subset    
# pool2subset('train001', 50)
# # Adds an additional 100 records 'train000' subset
# pool2subset('validate001', 50)
# print('done')

In [6]:
def runquery(sql: str):
    """ 
    This function facilitates ad hoc queries.
    """
    conn = sqlite3.connect(DB_PATH)
    results = conn.execute(sql).fetchall()
    conn.close()
    return results

# runquery('select count(*) from images GROUP BY subset;')

In [7]:
def create_dataset(database_path: str, dataset_path: str, image_dir_path: str):
    """
    Create a YOLO dataset containing all images referred to in the "obj" database table.
    """

    # delete folders and files if they exist

    dirpath = dataset_path
    if os.path.exists(dirpath) and os.path.isdir(dirpath):
        shutil.rmtree(dirpath)

    # create folder structure

    for dir in ['train', 'val']:
        os.makedirs(f'{dataset_path}/{dir}')

    # save data.yaml in top level folder
    
    s = f'train: {dataset_path}/train\n'
    s += f'val: {dataset_path}/val\n'
    s += 'nc: 6\n'
    s += 'names: [zero, low, medium, high, dead, vcut]\n'
    
    with open(f'{dataset_path}/data.yaml', 'w') as f:
        f.write(s)
        
    # save classes.txt in train and val folders

    s = 'zero\n'
    s += 'low\n'
    s += 'medium\n'
    s += 'high\n'
    s += 'dead\n'
    s += 'vcut\n'
    
    with open(f'{dataset_path}/train/classes.txt', 'w') as f:
        f.write(s)
    with open(f'{dataset_path}/val/classes.txt', 'w') as f:
        f.write(s)
        
    # get data from database: select all records from "obj" table and get add "img.subset"

    sql = 'SELECT img.subset, obj.* FROM obj LEFT JOIN img ON img.filename = obj.filename'
    df = pd.read_sql_query(sql, sqlite3.connect(database_path))

    for filename in df.filename.unique():
        
        # create and save a labels file
        
        labels = []
        df1 = df[df.filename == filename]   # create dataframe for objects detected in filename
        subset = df1.iloc[0]['subset']
        s = ''
        for i, r in df1.iterrows():
            s += f'{r["cls"]} {r["x"]:.6f} {r["y"]:.6f} {r["w"]:.6f} {r["h"]:.6f}\n'
        labels_file_path = f'{dataset_path}/{subset}/{filename.replace("jpg", "txt")}'
        with open(labels_file_path, 'w') as f:
                f.write(s)
                
        # create and save symlink to image file
        
        src = f'{image_dir_path}/{filename}'
        dst = f'{dataset_path}/{subset}'
        subprocess.run(['ln', '-s', src, dst])   
        
# create_dataset(DB_PATH, DATASET_PATH, IMAGE_DIR_PATH)   

In [8]:
def annotate(subset: str):
    """
    Executes labelImg to annotate specified subset
    """
    subprocess.run([
        'python3', 
        '/home/aubrey/labelImg/labelImg.py',
        f'/home/aubrey/Desktop/Guam07-training-set/datasets/active_learning/{subset}'])   
    
# annotate('validate000')

In [9]:
def detect_objects(model_file_path: str, image_dir_path: str):
    model = YOLO(MODEL_FILE_PATH)
    results = model(
        source=f'{image_dir_path}/*.jpg',
        imgsz=960,
        agnostic_nms=True,
        iou=0.5,
        stream=True,
        )
    return results

In [10]:
def detect_and_store_objects(model_file_path: str, image_dir_path, db_path: str):
    """
    Use a pretrained model to detect objects in all images stored in image_dir_path and save results in the 'obj' database table.
    Note that no data are stored for negative images (images in which objects are not detected).
    Also note that all records in the obj table are replaced by this function.
    """
    results = detect_objects(model_file_path, image_dir_path)

    # update the obs database table
    conn = sqlite3.connect(db_path)
    conn.execute('DELETE FROM obj;')
    conn.commit()
    
    for r in results:
        filename = os.path.basename(r.path)
        boxes = r.boxes.cpu().numpy()
        class_ids = boxes.cls
        confidences = boxes.conf
        xywhns = boxes.xywhn
        n = len(class_ids)
        for i in range(n):
            sql = f'''INSERT INTO obj (filename, cls, x, y, w, h, conf)
            VALUES ("{filename}", {class_ids[i]}, 
                {xywhns[i][0]}, {xywhns[i][1]}, {xywhns[i][2]}, {xywhns[i][3]},
                {confidences[i]});'''
            conn.execute(sql)
    conn.commit()
    conn.close()

In [11]:
def print_db_summary(db_path: str):
    conn = sqlite3.connect(DB_PATH)

    sql = 'SELECT COUNT(*) FROM img;'
    df = pd.read_sql_query(sql, conn)
    print(f'{sql}\n{df}\n')

    sql = 'SELECT COUNT(DISTINCT(filename)) FROM obj;'
    df = pd.read_sql_query(sql, conn)
    print(f'{sql}\n{df}\n')

    sql = 'SELECT COUNT(*) FROM obj;'
    df = pd.read_sql_query(sql, conn)
    print(f'{sql}\n{df}\n')

    sql = 'SELECT cls, count(*) FROM obj GROUP BY cls;'
    df = pd.read_sql_query(sql, conn)
    print(f'{sql}\n{df}\n')

    conn.close()
    
# print_db_summary(DB_PATH)


# MAIN

In [12]:
# MAIN

random.seed(42) # set seed to ensure reproducible random number sequences

In [13]:
# Create database if it does not already exist

if not os.path.exists(DB_PATH):
    create_db(IMAGE_DIR_PATH, DB_PATH)

In [14]:
# Use a pretrained model to detect all objects in images stored in IMAGE_DIR_PATH and save data in the "obj" database table.  
 
detect_and_store_objects(MODEL_FILE_PATH, IMAGE_DIR_PATH, DB_PATH)



image 1/26263 /home/aubrey/Desktop/Guam07-training-set/rawdata/IMG_20221115_111515.jpg: 544x960 (no detections), 81.6ms
image 2/26263 /home/aubrey/Desktop/Guam07-training-set/rawdata/IMG_20221115_111516.jpg: 544x960 (no detections), 5.5ms
image 3/26263 /home/aubrey/Desktop/Guam07-training-set/rawdata/IMG_20221115_111517.jpg: 544x960 (no detections), 5.5ms
image 4/26263 /home/aubrey/Desktop/Guam07-training-set/rawdata/IMG_20221115_111518.jpg: 544x960 (no detections), 4.8ms
image 5/26263 /home/aubrey/Desktop/Guam07-training-set/rawdata/IMG_20221115_111519.jpg: 544x960 (no detections), 4.1ms
image 6/26263 /home/aubrey/Desktop/Guam07-training-set/rawdata/IMG_20221115_111521.jpg: 544x960 (no detections), 4.3ms
image 7/26263 /home/aubrey/Desktop/Guam07-training-set/rawdata/IMG_20221115_111522.jpg: 544x960 (no detections), 4.0ms
image 8/26263 /home/aubrey/Desktop/Guam07-training-set/rawdata/IMG_20221115_111523.jpg: 544x960 (no detections), 4.5ms
image 9/26263 /home/aubrey/Desktop/Guam07-trai

In [15]:
print_db_summary(DB_PATH)

SELECT COUNT(*) FROM img;
   COUNT(*)
0     26263

SELECT COUNT(DISTINCT(filename)) FROM obj;
   COUNT(DISTINCT(filename))
0                      10416

SELECT COUNT(*) FROM obj;
   COUNT(*)
0     23128

SELECT cls, count(*) FROM obj GROUP BY cls;
   cls  count(*)
0    0      3409
1    1     11974
2    2      2494
3    3       588
4    4       241
5    5      4422



## Create YOLO dataset for further editing and training

In [16]:
n_images = runquery('SELECT COUNT(*) FROM img;')[0][0]
n_train = int(0.9 * n_images)
pool2subset('train', n_train)
pool2subset('val', 1000000) # convert all remaining 'pool' records to 'val'

06:42:40 --> 2671457274.py:6 in pool2subset() at 06:42:40.580


06:42:40 --> 2671457274.py:6 in pool2subset() at 06:42:40.607


In [17]:
conn = sqlite3.connect(DB_PATH)
sql = 'select subset, count(*) as n  from img group by subset;'
pd.read_sql_query(sql, conn)

,subset,n
0,train,23636
1,val,2627


In [18]:
# Create a YOLO dataset for further editing and training

create_dataset(DB_PATH, DATASET_PATH, IMAGE_DIR_PATH)

In [19]:
ic('FINISHED')

06:43:02 --> 'FINISHED'


'FINISHED'

In [20]:
    



# ic(runquery('SELECT subset, COUNT(*) FROM images GROUP BY subset;'))
# pool2subset('train000', 50)
# pool2subset('validate000', 50)
# ic(runquery('SELECT subset, COUNT(*) FROM images GROUP BY subset;'))

# # Create dataset

# create_dataset()

# # Annotate train000

# # annotate('train000')

# ic('FINISHED')
# ic('Run "annotate(train000) to annotate train000 using labelImg,')

In [2]:
from ultralytics import YOLO

# resume training

model = YOLO('/home/aubrey/Desktop/Guam07-training-set/code/runs/detect/imgsz9603/weights/best.pt')

results = model.train(
    data="/home/aubrey/Desktop/Guam07-training-set/datasets/active_learning2/data.yaml", 
    epochs=1000,
    imgsz=960,
    batch=-1, 
    name='imgsz960',
    )

print('FINISHED TRAINING')

New https://pypi.org/project/ultralytics/8.2.56 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.14 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 16110MiB)
engine/trainer: task=detect, mode=train, model=/home/aubrey/Desktop/Guam07-training-set/code/runs/detect/imgsz9603/weights/best.pt, data=/home/aubrey/Desktop/Guam07-training-set/datasets/active_learning2/data.yaml, epochs=1000, time=None, patience=100, batch=-1, imgsz=960, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=imgsz96011, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=No

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

train: Scanning /home/aubrey/Desktop/Guam07-training-set/datasets/active_learning2/train... 9396 images, 43 backgrounds, 0 corrupt: 100%|██████████| 9396/9396 [00:03<00:00, 2413.86it/s]


train: New cache created: /home/aubrey/Desktop/Guam07-training-set/datasets/active_learning2/train.cache


...

   101/1000      9.65G     0.7488     0.6898      1.013         59        960: 100%|██████████| 294/294 [01:58<00:00,  2.49it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:04<00:00,  3.68it/s]

                   all       1018       2254      0.752      0.706      0.795      0.636
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 1, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



101 epochs completed in 3.429 hours.
Optimizer stripped from runs/detect/imgsz96011/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/imgsz96011/weights/best.pt, 6.3MB

Validating runs/detect/imgsz96011/weights/best.pt...
Ultralytics YOLOv8.2.14 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 16110MiB)
Model summary (fused): 168 layers, 3006818 parameters, 0 gradients, 8.1 GFLOPs


/home/aubrey/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:05<00:00,  3.18it/s]


                   all       1018       2254      0.782      0.793      0.877      0.753
                  zero       1018        348      0.742      0.779      0.853      0.751
                   low       1018       1146      0.672      0.969      0.921      0.807
                medium       1018        250       0.78      0.838      0.875      0.765
                  high       1018         54       0.78      0.796      0.885      0.791
                  dead       1018         27       0.94      0.586      0.853      0.679
                  vcut       1018        429      0.778      0.792      0.876      0.724
Speed: 0.2ms preprocess, 1.8ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/imgsz96011
FINISHED TRAINING
